# **NumberMind**

Installing all necessary libraries and imports.

In [1]:
! pip install python-sat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 13.7 MB/s 


In [2]:
import numpy as np
import itertools
np.random.seed(123)

In [3]:
from pysat.solvers import Minisat22

Firstly we generate a secret code which is the main focus of the game. The master selects a secret code and the player is the one who is supposed to guess it. In our case, the machine is playing a NumberMind game againsts itself. 


In [4]:
#We create a function for the secret code

def secret_code(n):
  secret_code=[]
  for i in range(n):
    secret_code.append(np.random.randint(0,10))

  #return secret_code
  return np.array(secret_code)


#We create a function that generates a random guess to be our first guess. It is okay for it to be random because it is the first trial and from there we continue.

def first_guess(n):
  return np.array([np.random.randint(0,10) for i in range(n)])

code=secret_code(10)

code


array([2, 2, 6, 1, 3, 9, 6, 1, 0, 1])

Following we create a function that returns information on the correctly guessed digits of the guessing code. We consider that the digit is correct only if it is in the exact position w.r.t the secret code.

In [5]:
def check(code, guess):
  guess=np.array(guess)
  k=0
  for i in range(code.shape[0]):
    if guess[i]==code[i]:
      k=k+1

  return k

Function to encode with propositional language notation the 10*n propositional variables that make our secret code.

In [6]:
def language (n):
  p=dict()
  x=0
  for i in range(n):
    for j in range(10):
      x+=1
      p[i,j]=x

  return p

In [7]:
p = language(5)

In [8]:
p

{(0, 0): 1,
 (0, 1): 2,
 (0, 2): 3,
 (0, 3): 4,
 (0, 4): 5,
 (0, 5): 6,
 (0, 6): 7,
 (0, 7): 8,
 (0, 8): 9,
 (0, 9): 10,
 (1, 0): 11,
 (1, 1): 12,
 (1, 2): 13,
 (1, 3): 14,
 (1, 4): 15,
 (1, 5): 16,
 (1, 6): 17,
 (1, 7): 18,
 (1, 8): 19,
 (1, 9): 20,
 (2, 0): 21,
 (2, 1): 22,
 (2, 2): 23,
 (2, 3): 24,
 (2, 4): 25,
 (2, 5): 26,
 (2, 6): 27,
 (2, 7): 28,
 (2, 8): 29,
 (2, 9): 30,
 (3, 0): 31,
 (3, 1): 32,
 (3, 2): 33,
 (3, 3): 34,
 (3, 4): 35,
 (3, 5): 36,
 (3, 6): 37,
 (3, 7): 38,
 (3, 8): 39,
 (3, 9): 40,
 (4, 0): 41,
 (4, 1): 42,
 (4, 2): 43,
 (4, 3): 44,
 (4, 4): 45,
 (4, 5): 46,
 (4, 6): 47,
 (4, 7): 48,
 (4, 8): 49,
 (4, 9): 50}

For the purpose of doing propositional language decoding, we create a function that starting from a model returned by SAT turns it into a guess, so into a sequence of integers. Before doing this we need to create a key associated to the values in a dictionary. We do this by making a function to get the key. 

In [9]:
def get_key(d,val):
  for key, value in d.items():
    if val==value:
      return key

  return "key is not found"

In [10]:
def decoding(model):
  pos=[]
  for i in range(len(model)):
    if model[i]>0:
      pos.append(get_key(p,model[i]))
#To generate the sequence we have to put the positions (i,j) in ascending order w.r.t the j index 
  sequence=np.array([pos[i][1] for i in range(len(pos))])

  return sequence

In [11]:
def half_decode(guess):
  pos=[]
  for i in range(len(guess)):
    pos.append([i, guess[i]])
  return pos

In [12]:
half_decode([1,2,3,4])

[[0, 1], [1, 2], [2, 3], [3, 4]]

Function for defining the clauses. 

In [13]:
def clauses(n):
  y=Minisat22()
  #At least one 
  for i in range(n):
    y.add_clause([p[i,j] for j in range(10)])
  #At most one 
  for i in range(n):
    for j in range(10):
      for k in range(j+1, 10):
        y.add_clause([-p[i,j], -p[i,k]])

  return y

In [14]:
#To check

y=clauses(5)
y.solve()

True

In [15]:
k = 0
for m in y.enum_models():
  k=k+1

print("Number of possible codes: " + str(k))

Number of possible codes: 100000


Generating the new code

After a code guess we get a feedback in the form : "k correct digits" with (0<= k <= n).
- The first guess is generated at random and satisfies only the clauses.
- If k=n we have guessed correctly the secret code and the game is over.
- If 0 < k < n-1 instead we have to exploit the feedback obtained to generate a new guess. We have to add the constraints "exactly k are true" on the propositional variables selected by our code guess.

In [16]:
#function for subsets generation
def findsubsets(s,n):
  return list(itertools.combinations(s,n))

In [17]:
def generate (guess,idx):
  #guess gives k=n-1
  #idx is all the possible codes changing the number in index idx in guess
  list_guesses=[]
  for i in range(10):
    if guess[idx] == i: #in this case we are regenerating the starting guess that we know has 1 error so we skip it 
      continue 
    lg=list(guess)
    lg1=lg[:idx]
    lg1.append(i)
    lg2=lg1+lg[idx+1:]
    list_guesses.append(lg2)

  return list_guesses

generate(np.array([0,2,3]),0)

[[1, 2, 3],
 [2, 2, 3],
 [3, 2, 3],
 [4, 2, 3],
 [5, 2, 3],
 [6, 2, 3],
 [7, 2, 3],
 [8, 2, 3],
 [9, 2, 3]]

Implement the Game

In [20]:
num_it_ex=[]
for i in range(1):
  print()
  print('############## WELCOME TO NUMBER MIND GAME ################')
  print()
  ndig=int(input("Insert a number of digits to play with: "))
  code=secret_code(ndig)
  print('Secret code ' +str(code))
  print('Try to guess the secret code: ')
  p = language(ndig)
  y=clauses(ndig)
  num_it=0
  max_it=500
  guess=first_guess(ndig)
  k=check(code, guess)
  print('Guess ' + str(num_it) + ": " + str(guess) + ' -> ' + str(k) + ' correct digits, try again!')
  flag=False
  flag1=False
  while(True):
    if num_it==max_it:
      print('Too much iterations')
      break

    if k==ndig:
      print()
      print('############################# WOW, YOU WON! #########################')
      print()
      print('You did it in ' + str(num_it) + ' tries ')
      num_it_ex.append(num_it)
      break

    elif k==0 and flag1==False:
      guess=first_guess(ndig)
      k=check(code,guess)
      print('Guess ' + str(num_it+1) + ": " + str(guess) + ' -> ' + str(k) + ' correct digits, try again!')


    elif k==0 and flag1==True:
      j=0
      for m in y.enum_models():
        if j==1:
          break
        guess=np.array(decoding(m))
        j=j+1

      k=check(code,guess)
      print('Guess ' + str(num_it+1) + ": " + str(guess) + ' -> ' + str(k) + ' correct digits, try again!')


    elif k==ndig-1:
      #save the guess that makes k=n-1 in the variable guess_n1
         guess_n1=guess
         for i in range(ndig):
           if k==ndig:
             flag=True
             break 
           l_guess=generate(guess_n1,i)
           for j in range(len(l_guess)):
            guess=l_guess[j]
            k=check(code, guess)
            print('Guess ' +str(num_it+1) + ': ' + str(np.array(guess)) + ' -> ' + str(k) + ' correct digits, try again!')
            if k<ndig-1:
               num_it=num_it+1
               break 
            if k == ndig:
               break
            num_it=num_it+1

    elif k==ndig or flag==True:
              print()
              print('############################# WOW, YOU WON! #########################')
              print()

              print('You did it in ' + str(num_it-1) + ' tries')
              num_it_ex.append(num_it)
              break

    else: 
              flag1=True
              #at least k true
              #subsets od n-k+1 variables
              sub=findsubsets(half_decode(guess),ndig-k+1) #this sub should be a list of list
              for l in sub:
                y.add_clause([(p[tuple(i)]) for i in l])

              #at most k true
              sub1=findsubsets(half_decode(guess),k+1)  
              for l1 in sub1:
                y.add_clause([(-p[tuple(i)]) for i in l1])


              j=0
              for m in y.enum_models():
                if j==1:
                  break
                guess=np.array(decoding(m))
                j=j+1

              k=check(code,guess)
              print('Guess ' +str(num_it+1) + ': ' + str(np.array(guess)) + ' -> ' + str(k) + ' correct digits, try again!')

    num_it=num_it+1   


      






############## WELCOME TO NUMBER MIND GAME ################

Insert a number of digits to play with: 5
Secret code [9 3 4 6 1]
Try to guess the secret code: 
Guess 0: [5 6 2 1 8] -> 0 correct digits, try again!
Guess 1: [3 5 0 2 6] -> 0 correct digits, try again!
Guess 2: [2 4 4 6 3] -> 2 correct digits, try again!
Guess 3: [2 4 0 0 0] -> 0 correct digits, try again!
Guess 4: [2 4 0 1 1] -> 1 correct digits, try again!
Guess 5: [2 2 4 8 7] -> 1 correct digits, try again!
Guess 6: [2 0 5 0 3] -> 0 correct digits, try again!
Guess 7: [2 3 9 0 3] -> 1 correct digits, try again!
Guess 8: [5 8 4 1 3] -> 1 correct digits, try again!
Guess 9: [6 4 4 0 9] -> 1 correct digits, try again!
Guess 10: [0 4 5 8 3] -> 0 correct digits, try again!
Guess 11: [3 4 1 8 3] -> 0 correct digits, try again!
Guess 12: [0 4 7 8 3] -> 0 correct digits, try again!
Guess 13: [3 4 2 8 3] -> 0 correct digits, try again!
Guess 14: [0 4 6 8 3] -> 0 correct digits, try again!
Guess 15: [3 4 8 8 3] -> 0 correct digits